# Speeding up the process


In [15]:
import time

def check_value_in_list(x):
    for i in range(10**8):
        i in x

comparision_list = [1, 2, 3]
start_time = time.time()

check_value_in_list(comparision_list)
print('Everything took: ', time.time() - start_time, ' seconds')

Everything took:  4.515764474868774  seconds


## Threading

In [3]:
from threading import Thread
import time


def check_value_in_list(x):
    for i in range(10**8):
        i in x


num_threads = 4
comparision_list = [1, 2, 3]

start_time = time.time()
threads = []


for i in range(num_threads):
    t = Thread(target=check_value_in_list, args=(comparision_list,))
    threads.append(t)

for t in threads:
    t.start()

for t in threads:
    t.join()

print('Everything took: ', time.time() - start_time, ' seconds')

Everything took:  19.679632663726807  seconds


## Multiprocessing with Process

In [27]:
from multiprocessing import Process
import time


def check_value_in_list(x):
    for i in range(10**8):
        i in x


num_proccesses = 4
comparision_list = [1, 2, 3]

start_time = time.time()
proccesses = []


for i in range(num_proccesses):
    t = Process(target=check_value_in_list, args=(comparision_list,))
    threads.append(t)

for t in proccesses:
    t.start()

for t in proccesses:
    t.join()

print('Everything took: ', time.time() - start_time, ' seconds')

Everything took:  0.00020694732666015625  seconds


## Multiprocessing Queues

In [29]:
from multiprocessing import Process, Queue
import time


def check_value_in_list(x, i, num_proccesses, queue):
    max_number_to_check_to = 10**8
    lower = int(i * max_number_to_check_to / num_proccesses)
    upper = int((i+1) * max_number_to_check_to / num_proccesses)
    number_of_hits = 8
    for i in range(lower, upper):
        if i in x:
            number_of_hits += 1
    queue.put((lower, upper, number_of_hits))


num_proccesses = 4
comparision_list = [1, 2, 3]

queue = Queue()

start_time = time.time()
proccesses = []


for i in range(num_proccesses):
    t = Process(target=check_value_in_list, args=(comparision_list, i, num_proccesses, queue))
    threads.append(t)

for t in proccesses:
    t.start()

for t in proccesses:
    t.join()

queue.put('DONE')

while True:
    v = queue.get()
    if v == 'DONE':
        break
    lower, upper, number_of_hits = v
    print('Between', lower, 'and', upper, 'we have', number_of_hits, 'values in list')

print('Everything took: ', time.time() - start_time, ' seconds')

Everything took:  0.004309654235839844  seconds


## Using Multiprocessing Pool

In [36]:
from multiprocessing import Pool, cpu_count
import time

start_time = time.time()

def square(x):
    return x**2


comparision_list = [1, 2, 3]

num_cpu_to_use = max(1,cpu_count() -1)
print('number of cpu in use:',num_cpu_to_use)

with Pool(processes=num_cpu_to_use) as mp_pool: #we can define number of processes
    result = mp_pool.map(square, comparision_list)

print(result)

print('Everything took: ', time.time() - start_time, ' seconds')

number of cpu in use: 11
[1, 4, 9]
Everything took:  0.09054327011108398  seconds


## Passing Multiple values to Pool

In [39]:
from multiprocessing import Pool, cpu_count
from functools import partial #partially defined functions
import time

start_time = time.time()

def square(y, additional, x): #
    return x**y + additional


comparision_list = [1, 2, 3]
power = 6
additional = 2
num_cpu_to_use = max(1,cpu_count() -1)

print('number of cpu in use:',num_cpu_to_use)

partial_function = partial(square, power, additional)

with Pool(processes=num_cpu_to_use) as mp_pool: #we can define number of processes
    result = mp_pool.map(partial_function, comparision_list)

print(result)

print('Everything took: ', time.time() - start_time, ' seconds')

number of cpu in use: 11
[3, 66, 731]
Everything took:  0.11655211448669434  seconds


## Multiprocessing Multiple Varying Arguments

In [43]:
from multiprocessing import Pool, cpu_count
from functools import partial #partially defined functions
import time

start_time = time.time()

def square(x, y): #passing in a tuple consisting of variying arguments
    return x**y 

comparision_list = [1,2,3]
power_list = [4,5,6]

num_cpu_to_use = max(1,cpu_count() -1)

print('number of cpu in use:',num_cpu_to_use)

prepared_list = list(zip(comparision_list, power_list))
print(prepared_list)
with Pool(processes=num_cpu_to_use) as mp_pool: 
    result = mp_pool.starmap(square, prepared_list)

print(result)

print('Everything took: ', time.time() - start_time, ' seconds')

number of cpu in use: 11
[(1, 4), (2, 5), (3, 6)]
[1, 32, 729]
Everything took:  0.09872579574584961  seconds


## Multiprocessing Checking Elements in List in Certain Ranges

In [51]:
from multiprocessing import Pool, cpu_count
import time

start_time = time.time()


def check_value_in_list(comp_list, lower, upper):
    number_of_hits = 8
    for i in range(lower,upper):
        if i in comp_list:
            number_of_hits += 1
    return number_of_hits

comparision_list = [1,2,3]
lower_and_upper_bounds = [(0, 25*10**6),(25*10**6, 50*10**6)
                          ,(50*10**6, 75*10**6),(75*10**6, 100*10**6)]
num_cpu_to_use = max(1,cpu_count() -1)

print('number of cpu in use:',num_cpu_to_use)

prepared_list = []
for i in range(len(lower_and_upper_bounds)):
    prepared_list.append((comparision_list,*lower_and_upper_bounds[i])) # unpack the tuples

print(prepared_list)
with Pool(processes=num_cpu_to_use) as mp_pool: 
    result = mp_pool.starmap(check_value_in_list, prepared_list)

print(result)

print('Everything took: ', time.time() - start_time, ' seconds')

number of cpu in use: 11
[([1, 2, 3], 0, 25000000), ([1, 2, 3], 25000000, 50000000), ([1, 2, 3], 50000000, 75000000), ([1, 2, 3], 75000000, 100000000)]
[11, 8, 8, 8]
Everything took:  1.7143595218658447  seconds
